In [32]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from scipy.spatial.distance import cdist
import numpy as np
import pm4py

In [33]:
rtfm_raw=pm4py.read_xes("./Data/road_traffic/RawData/Road_Traffic_Fine_Management_Process.xes")

c:\Users\ccagu\anaconda3\envs\declareTrees\lib\site-packages\pm4py\util\dt_parsing\parser.py:77: UserWarning: ISO8601 strings are not fully supported with strpfromiso for Python versions below 3.11
  warnings.warn(
parsing log, completed traces :: 100%|██████████| 150370/150370 [00:35<00:00, 4283.01it/s]


In [ ]:
dataset=pd.read_csv("./Data/road_traffic/mined_rtfm_relabelled_confidences.csv",index_col=0)


In [15]:
dataset = dataset.set_index("case:concept:name")

In [16]:
X=dataset.drop(columns=["Class"])

y=dataset['Class']
print("No. of features:"+str(len(X.columns)))

No. of features:2189


In [17]:
le = LabelEncoder()
print("Is na? "+str(X.isnull().values.any()))
y_transformed = le.fit_transform(y)
le_name_mapping = dict(zip(le.classes_,le.transform(le.classes_)))
cols=X.columns.to_list()
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y_transformed,
                                                    test_size=0.2,
                                                    stratify=y_transformed,
                                                    shuffle=True,#disorder the data
                                                    random_state=0)

Is na? True


In [26]:
le.classes_

array(['credit_collection', 'dismissed', 'paid_full', 'unresolved'],
      dtype=object)

In [25]:
le.transform(le.classes_)

array([0, 1, 2, 3])

In [36]:
def calculateBoundaryCasesThroughDistance(data, class1, class2, y):
    data=data.fillna(-100)
    data_class1=data[y==class1].drop_duplicates()
    data_class2=data[y==class2].drop_duplicates()
    matrixDistances=cdist(data_class1, data_class2)
    indexMinDistance=np.argmin(matrixDistances)#we obtain the index of the minimum distance
                        
    #Transform the index of the minimum distance to real index:
    min_idx = np.unravel_index(indexMinDistance, matrixDistances.shape)

    #The x component represents a case of classX, and the y component represents a case of classY, so we filter them:
    min_case_class1=data_class1.loc[[list(data_class1.index)[min_idx[0]]]]
    min_case_class2=data_class2.loc[[list(data_class2.index)[min_idx[1]]]]

    return (min_case_class1, min_case_class2)




In [37]:
boundaryCasesClass1vs3=calculateBoundaryCasesThroughDistance(X_test, 1, 3, y_test)

In [40]:
rtfm_raw[rtfm_raw["case:concept:name"]==boundaryCasesClass1vs3[0].index[0]]

,amount,org:resource,dismissal,concept:name,vehicleClass,totalPaymentAmount,lifecycle:transition,time:timestamp,article,points,case:concept:name,expense,notificationType,lastSent,paymentAmount,matricola
516599,33.60,843,NIL,Create Fine,A,0.0,complete,2003-08-07 00:00:00+00:00,158.0,0.0,S88125,NaN,NaN,NaN,NaN,NaN
516600,NaN,NaN,NaN,Send Fine,NaN,NaN,complete,2003-11-03 00:00:00+00:00,NaN,NaN,S88125,10.0,NaN,NaN,NaN,NaN
516601,NaN,NaN,NaN,Insert Fine Notification,NaN,NaN,complete,2003-11-13 00:00:00+00:00,NaN,NaN,S88125,NaN,P,N,NaN,NaN
516602,68.77,NaN,NaN,Add penalty,NaN,NaN,complete,2004-01-12 00:00:00+00:00,NaN,NaN,S88125,NaN,NaN,NaN,NaN,NaN
516603,NaN,NaN,NaN,Insert Date Appeal to Prefecture,NaN,NaN,complete,2004-01-12 00:00:00+00:00,NaN,NaN,S88125,NaN,NaN,NaN,NaN,NaN
516604,NaN,NaN,#,Send Appeal to Prefecture,NaN,NaN,complete,2004-02-12 00:00:00+00:00,NaN,NaN,S88125,NaN,NaN,NaN,NaN,NaN


In [41]:
rtfm_raw[rtfm_raw["case:concept:name"]==boundaryCasesClass1vs3[1].index[0]]

,amount,org:resource,dismissal,concept:name,vehicleClass,totalPaymentAmount,lifecycle:transition,time:timestamp,article,points,case:concept:name,expense,notificationType,lastSent,paymentAmount,matricola
50317,36.0,562,NIL,Create Fine,A,0.0,complete,2008-08-06 00:00:00+00:00,157.0,0.0,A34464,NaN,NaN,NaN,NaN,NaN
50318,NaN,NaN,NaN,Send Fine,NaN,NaN,complete,2008-10-21 00:00:00+00:00,NaN,NaN,A34464,13.5,NaN,NaN,NaN,NaN
50319,NaN,NaN,NaN,Insert Fine Notification,NaN,NaN,complete,2008-10-29 00:00:00+00:00,NaN,NaN,A34464,NaN,P,P,NaN,NaN
50320,74.0,NaN,NaN,Add penalty,NaN,NaN,complete,2008-12-28 00:00:00+00:00,NaN,NaN,A34464,NaN,NaN,NaN,NaN,NaN
50321,NaN,NaN,NaN,Insert Date Appeal to Prefecture,NaN,NaN,complete,2009-01-17 00:00:00+00:00,NaN,NaN,A34464,NaN,NaN,NaN,NaN,NaN
50322,NaN,NaN,NIL,Send Appeal to Prefecture,NaN,NaN,complete,2009-02-09 00:00:00+00:00,NaN,NaN,A34464,NaN,NaN,NaN,NaN,NaN


In [ ]:
#2->3

In [42]:
boundaryCasesClass2vs3=calculateBoundaryCasesThroughDistance(X_test, 2, 3, y_test)

In [43]:
rtfm_raw[rtfm_raw["case:concept:name"]==boundaryCasesClass2vs3[0].index[0]]

,amount,org:resource,dismissal,concept:name,vehicleClass,totalPaymentAmount,lifecycle:transition,time:timestamp,article,points,case:concept:name,expense,notificationType,lastSent,paymentAmount,matricola
151921,32.8,537,NIL,Create Fine,A,0.0,complete,2001-11-09 00:00:00+00:00,157.0,0.0,N38303,NaN,NaN,NaN,NaN,NaN
151922,NaN,NaN,NaN,Payment,NaN,32.8,complete,2001-11-13 00:00:00+00:00,NaN,NaN,N38303,NaN,NaN,NaN,32.8,NaN


In [44]:
rtfm_raw[rtfm_raw["case:concept:name"]==boundaryCasesClass2vs3[1].index[0]]

,amount,org:resource,dismissal,concept:name,vehicleClass,totalPaymentAmount,lifecycle:transition,time:timestamp,article,points,case:concept:name,expense,notificationType,lastSent,paymentAmount,matricola
109818,31.3,550,NIL,Create Fine,A,0.00,complete,2000-01-20 00:00:00+00:00,157.0,0.0,N22520,NaN,NaN,NaN,NaN,NaN
109819,NaN,NaN,NaN,Payment,NaN,30.99,complete,2000-02-04 00:00:00+00:00,NaN,NaN,N22520,NaN,NaN,NaN,30.99,NaN
